In [1]:
! pip install xgboost

     ---------------------------------------- 99.7/99.7 MB 1.6 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np

In [3]:

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [4]:
train_two = []
cur =0
for i in range(train_data.shape[0]):
  z  = len(train_data.iloc[i][0])
  x =0
  for j in range(z):
    y = (ord(train_data.iloc[i][0][j])-65)
    if((y<0 or y>25)==True):
      x=1
      break
  if(x==0):
    train_two.append(train_data.iloc[i])
    cur+=1


train_two = pd.DataFrame(train_two)
train_data = train_two

In [5]:
train_label = train_data['Label']
train_feature20 = []
test_feature20=[]


for i in range(train_data.shape[0]):
  x = []
  for j in range(26):
    x.append(0)
  z = len(train_data.iloc[i][0])
  for j in range(z):
    y = (ord(train_data.iloc[i][0][j])-65)
    x[y]=x[y]+1

  for j in range(26):
    x[j]=x[j]/z


  train_feature20.append(x)


for i in range(test_data.shape[0]):
  x = []
  for j in range(26):
    x.append(0)
  z = len(test_data.iloc[i][1])
  for j in range(z):
    y = (ord(test_data.iloc[i][1][j])-65)
    x[y]=x[y]+1
  for j in range(26):
    x[j]=x[j]/z
  test_feature20.append(x)


test_feature20 = pd.DataFrame(test_feature20)
train_feature20 = pd.DataFrame(train_feature20)

In [8]:
test_feature2020 = []
test_feature2020 = pd.DataFrame(test_feature2020)
train_feature2020 = []
train_feature2020 = pd.DataFrame(train_feature2020)

test_feature2020 = test_feature20[0]
train_feature2020 = train_feature20[0]

for i in range(1,26):
  if(i!=1 and i!=23 and i!=25 and i!=(ord('J')-65) and i!=(ord('O')-65) and i!=(ord('U')-65)):
    test_feature2020 = pd.concat([test_feature2020,test_feature20[i]],axis=1)
    train_feature2020 = pd.concat([train_feature2020,train_feature20[i]],axis=1)

arr = []
notar = ['B','X','Z','O','J','U']

notadd = []

posa =[]
for i in range(26):
  for j in range(6):
    notadd.append(i*26+(ord(notar[j])-65))
    notadd.append((ord(notar[j])-65)*26+i)

In [9]:
train_feature = []
test_feature=[]


for i in range(train_data.shape[0]):
  x = []
  for j in range(26*26):
    x.append(0)
  z = len(train_data.iloc[i][0])
  for j in range(z-1):
    y = (ord(train_data.iloc[i][0][j])-65)
    p = (ord(train_data.iloc[i][0][j+1])-65)
    pos = ((y)*26) + p

    x[pos]=x[pos]+1

  for j in range(26*26):
    x[j]=x[j]/z

  train_feature.append(x)

for i in range(test_data.shape[0]):
  x = []
  for j in range(26*26):
    x.append(0)
  z = len(test_data.iloc[i][1])
  for j in range(z-1):
    y = (ord(test_data.iloc[i][1][j])-65)
    p = (ord(test_data.iloc[i][1][j+1])-65)
    pos = ((y)*26)+p

    x[pos]=x[pos]+1
  for j in range(26*26):
    x[j]=x[j]/z

  test_feature.append(x)

test_feature = pd.DataFrame(test_feature)
train_feature = pd.DataFrame(train_feature)


In [10]:
import pandas as pd
test_feature1 = test_feature[0]
train_feature1 = train_feature[0]

for i in range(1,26*26):
  if((i in notadd) == False):
    test_feature1 = pd.concat([test_feature1,test_feature[i]],axis=1)
    train_feature1 = pd.concat([train_feature1,train_feature[i]],axis=1)

In [11]:
from sklearn.feature_selection import SelectKBest, f_classif
import xgboost as xgb 
k = 290
# Feature selection
i=0
y_final=[]
# Feature selection
selector = SelectKBest(f_classif, k=k)
train_feature1_selected = selector.fit_transform(train_feature1, train_label)
test_feature1_selected = selector.transform(test_feature1)

while(i!=1):
    from sklearn import metrics
    # Create an XGBoost classifier
    xgb_classifier = xgb.XGBClassifier(
        n_estimators=100,  # You can adjust this parameter
        max_depth=3,        # You can adjust this parameter
        random_state=0
    )
    # Fit the classifier to the selected training features
    xgb_classifier.fit(train_feature1_selected, train_label)
# Make predictions on the selected testing features
    y_pred = xgb_classifier.predict_proba(test_feature1_selected)
    y_final.append(y_pred)
    i+=1


In [12]:

ans=[]
for i in range(310):
  ct =0
  for j in range(1):
    ct+=y_final[j][i][1]
  avg = ct/1
  ans.append(avg)

import pandas as pd
df = pd.DataFrame(ans)
df.columns = ['Label']
ID = np.arange(310) + 501
ID = pd.DataFrame(ID)
ID.columns = ['ID']
final_data = pd.concat([ID, df], axis=1)
final_data.to_csv('result3.csv', index=False)